# Modeling

## Prepare Dataset

In [1]:
import pandas as pd

In [2]:
train_path = r'./train_data_QUICK_START.csv'

In [3]:
train_df = pd.read_csv(train_path)

In [4]:
train_df.head()

,sequence_id,sequence,experiment_type,dataset_name,reactivity_0001,reactivity_0002,reactivity_0003,reactivity_0004,reactivity_0005,reactivity_0006,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,2A3_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,DMS_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,2A3_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,DMS_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00021f968267,GGGAACGACUCGAGUAGAGUCGAAAACAUUGUUAAUGCCUAUAUUA...,2A3_MaP,DasLabBigLib_OneMil_Replicates_from_previous_l...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, test_size=0.2, random_state=283)

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [8]:
max_seq_length = 206
nucleotides = 'ACGU'

In [11]:
def str_to_seq(s):
    mapping = {nucleotide: idx for idx, nucleotide in enumerate(nucleotides)}
    return [mapping[c] for c in s]


class RNADataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.label_idx = [idx for idx, column in enumerate(self.df.columns) if not column.startswith('reactivity_error') and column.startswith('reactivity')]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        input_seq = self.df['sequence'].iloc[idx]
        input_seq = str_to_seq(input_seq)
        input_seq = torch.LongTensor(input_seq)
        input_seq = F.one_hot(input_seq, num_classes=len(nucleotides))
        input_seq = input_seq.float()
        input_seq = F.pad(input_seq, pad=(0, 0, 0, max_seq_length - input_seq.size(0)))
        
        label_seq = self.df.iloc[idx, self.label_idx]
        label_seq = torch.FloatTensor(label_seq)
        label_seq = torch.nan_to_num(label_seq)
        label_seq = F.pad(label_seq, pad=(0, max_seq_length - label_seq.size(0)))
        
        return input_seq, label_seq

In [12]:
batch_size = 128

In [13]:
train_loader = DataLoader(RNADataset(train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(RNADataset(val), batch_size=batch_size, shuffle=True)

In [ ]:
for inputs, labels in train_loader:
    break

## Define and Train Model

In [20]:
from tqdm import tqdm
import numpy as np
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [39]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 300):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
    
class RNAModel(nn.Module):
    def __init__(self, embed_dim, d_model=128, dropout=0.1):
        super().__init__()

        self.pos = PositionalEncoding(4, dropout)
        self.conv = nn.Conv1d(embed_dim, d_model, kernel_size=3, padding=1)
        self.te = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=4, batch_first=True),
            num_layers=4
        )
        self.linear = nn.Linear(d_model, 1)
        
    def forward(self, x):
        x = self.pos(x)
        x = self.conv(x.transpose(-1, -2)).transpose(-1, -2)
        x = self.te(x)
        x = self.linear(x)
        
        return x.squeeze()

In [40]:
model = RNAModel(embed_dim=4).to(device)

criterion = nn.MSELoss()
mae = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

num_epochs = 100

In [41]:
loss_train = np.zeros(num_epochs)
loss_valid = np.zeros(num_epochs)
loss_mae_valid = np.zeros(num_epochs)

for epoch in tqdm(range(num_epochs)):
    train_size = 0
    valid_size = 0

    # train for one epoch
    model.train()
    for inputs, labels in train_loader:
        n_batch_train = len(inputs)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_train[epoch] += loss.item() * n_batch_train
        train_size += n_batch_train
    
    # validation
    with torch.no_grad():
        model.eval()
        for inputs, labels in val_loader:
            n_batch_valid = len(inputs)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss_valid[epoch] += loss.item() * n_batch_valid
            mae_loss = mae(outputs, labels)
            loss_mae_valid[epoch] += mae_loss.item() * n_batch_valid
            valid_size += n_batch_valid
    
    loss_train[epoch] /= train_size
    loss_valid[epoch] /= valid_size
    loss_mae_valid[epoch] /= valid_size
    print("epoch {}, train loss : {}, valid loss : {}, valid mae : {}".format(epoch, loss_train[epoch], loss_valid[epoch], loss_mae_valid[epoch]))

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\lingj\AppData\Local\Temp\ipykernel_25264\1738717559.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_seq = torch.FloatTensor(label_seq)
  1%|          | 1/100 [07:02<11:36:54, 422.37s/it]

epoch 0, train loss : 0.2065541178233138, valid loss : 0.19992104142160155, valid mae : 0.24751421290173248


  2%|▏         | 2/100 [14:01<11:26:31, 420.32s/it]

epoch 1, train loss : 0.20193038096417865, valid loss : 0.19878323102148687, valid mae : 0.2468419406467808


  3%|▎         | 3/100 [21:02<11:19:49, 420.51s/it]

...

epoch 97, train loss : 0.18031178521935584, valid loss : 0.1922840747606427, valid mae : 0.23394097860905433


 99%|█████████▉| 99/100 [11:27:57<06:54, 414.76s/it]

epoch 98, train loss : 0.18027984895025337, valid loss : 0.19189786201072218, valid mae : 0.23574475641419254


100%|██████████| 100/100 [11:34:53<00:00, 416.93s/it]

epoch 99, train loss : 0.18026172965083467, valid loss : 0.1916098181160746, valid mae : 0.23694010899931603
